In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])
     

gs


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# 1. 문서 로딩
print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader("C:/mylangchain/langchain_basic/data/tutorial-korean.pdf")  # 절대경로로 수정
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

# 2. 문서 분할
print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

# 3. 벡터화
print("==> 3. 벡터화 → 로컬 임베딩으로 변환")
embeddings = OllamaEmbeddings(model="qwen:3b")

# 4. 벡터스토어 저장
print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)

# 5. 검색기 생성
print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

# 6. LLM 생성기 설정
print("===> 6. 생성 → 로컬 LLM으로 답변 생성")
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434",
    temperature=0.1,
    num_predict=1500  # max_tokens → num_predict
)

# 7. QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 8. 예시 질문 실행
query = "이 튜토리얼에서 핵심 개념은 무엇인가요?"
result = qa_chain.invoke({"query": query})

print("\n[질문]")
print(query)
print("\n[답변]")
print(result["result"])


==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료
==> 2. 문서 분할 → 작은 청크로 나누기
  66개 청크 생성 완료
==> 3. 벡터화 → 로컬 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장


C:\Users\user\AppData\Local\Temp\ipykernel_8244\1294680578.py:26: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="qwen:3b")


ValueError: Error raised by inference API HTTP code: 404, {"error":"model \"qwen:3b\" not found, try pulling it first"}